In [1]:
from IPython.display import display, HTML
from fasthtml.common import *
from fasthtml.jupyter import *

def create_server(app,rt):
    if IN_NOTEBOOK:
        for port in range(8000,8030):   
            if 'server' in globals():
                print(f"Server already running on port {globals()['server'].port} - stopping it")
                globals()['server'].stop()
            if is_port_free(port):
                server = JupyUvi(app, port=port)

                def HShow(comp, app):
                    @app.get('/')
                    def get(): return comp
                    display(HTML(f'<a href="http://localhost:{port}/" target="_blank">Open in new tab</a>'))
                    return HTMX("/",port=port)
                print(f"Starter server on port {port}")
                Show = partial(HShow, app=app)
                return app, rt, server, HShow, Show
app, rt = fast_app()
app, rt, server, HShow, Show = create_server(app, rt)

Starter server on port 8002


In [2]:
def Reactive(tag: str = "div", **kwargs):
    id = unqid()

    imports = """
    import { createSignal } from 'https://esm.sh/solid-js@1.8.1';
    import { render } from 'https://esm.sh/solid-js@1.8.1/web';
    import h from 'https://esm.sh/solid-js@1.8.1/h'\n\n"""

    content = """const %s = () => {
         const [count, setCount] = createSignal(0);
         const incr = (e) => setCount(c => c+1);\n\n
         """ % tag.title()

    element = """return h('%s', {onClick: incr}, count);\n}\n""" % tag

    target = """render(Button, document.querySelector('#%s'));\n\n""" % id

    return (
        Div(id=id),
        Script(
            imports + content + element + target,
            type="module",
        ),
    )

In [3]:
@rt('/')
def get():
    return Button('Swap me', hx_get="/swap", hx_swap="outerHTML")

@rt('/swap')
def get():
    return Reactive('button')

HTMX()